# 라이브러리 import

In [179]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import rc # 폰트
from datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

pd.set_option ('display.max_rows', 10)



path = 'C:\\Users\\wldbs\\Desktop\\kart-game-project\\[data]_crawling\\'

# 크롤링 데이터 전처리

In [2]:
crawl = pd.read_csv(path + '01_crawling_track.csv', encoding = 'cp949')

In [3]:
# merge를 위해 변수명 변경

crawl.columns = ['map_name', '태그', '난이도', '랩', '트랙 길이', '진행 방향', '트랙 분류', 'AI 주행', '첫 등장일','리그 트랙', '라이센스', '약칭']

In [4]:
# 차이나 북경 경기장의 경우 난이도, 랩 잘못 입력되어있음. 아예 삭제하는 게 낫겠다는 판단
drop_data = crawl[crawl['난이도'].isnull()].index

crawl.drop(drop_data, inplace = True)

In [5]:
# 결측치 대치 (missing 이라는 문자값으로 입력되게 함)

crawl = crawl.fillna('missing')

In [6]:
# 필요 없는 컬럼 제거

crawl = crawl.drop(['태그', '트랙 분류', '리그 트랙', '라이센스', '약칭', 'AI 주행'], axis = 1)

In [7]:
# 난이도 문자열 - 숫자열로 변경

crawl['난이도'] = crawl.난이도.str.split('[').str[0]
crawl['난이도'] = crawl.난이도.str.split(' ').str[0]

crawl.난이도.unique()

crawl['난이도'].replace( {'●○○○○' : 1,
                          '●●○○○' : 2,
                          '●●●○○' : 3,
                          '●●●●○' : 4,
                          '●●●●●' : 5,
                          '●●○○○○' : 2,
                          '●●●●●○' : 4,
                          '●●●○○○' : 3,
                          '●●●●●●' : 5
                          }, inplace = True)

In [8]:
# 랩 문자열 - 숫자열로 변경

crawl['랩'] = crawl.랩.str.split('바').str[0]
crawl['랩']= crawl.랩.astype('int')

In [9]:
# 트랙 길이 문자열 - 숫자열로 변경

crawl['트랙 길이'].unique()

crawl['트랙 길이'] = crawl['트랙 길이'].str.split('k').str[0]
crawl['트랙 길이'] = crawl['트랙 길이'].str.split(' ').str[0]
crawl['트랙 길이'].replace( {'???' : 0 }, inplace = True)

crawl['트랙 길이'] = crawl['트랙 길이'].astype('float')

In [10]:
# 범주형 변수 따로 변환하지 않기로 함
crawl['진행 방향'].unique()

array(['반시계방향', '시계방향', '이중 8자형', '단선형', '8자형', '왕복형', '워프형', '스피드전',
       '삼중 8자형', '이중 시계방향', '이중 반시계방향', '복합 8자형'], dtype=object)

In [12]:
# # 날짜형 변환

crawl['년'] = crawl['첫 등장일'].str.split(' ').str[0]
crawl['월'] = crawl['첫 등장일'].str.split(' ').str[1]
crawl['일'] = crawl['첫 등장일'].str.split(' ').str[2]

crawl['년'] = crawl['년'].str.strip('년').str[0:4]
crawl['월'] = crawl['월'].str.split('월').str[0]
crawl['일'] = crawl['일'].str.split('일').str[0]

crawl['월'] = crawl['월'].str.rjust(width=2,fillchar='0')        # rlpad
crawl['일'] = crawl['일'].str.rjust(width=2,fillchar='0')        # rlpad

In [13]:
crawl['년'].unique()

array(['2004', '2006', '2007', '2009', '2016', '2005', '2019', '2010',
       '2015', '2020', '2008', 'miss', '2017', '2012', '2011', '2021',
       '2013', '2014', '200-', '2022', '-', '2018'], dtype=object)

In [14]:
# 년 변환
crawl['년'].replace( {'200-' : np.NaN,
                      'miss' : np.NaN,
                      '-'    : np.NaN}, inplace = True)

# 월 변환
crawl['월'].replace( {'0-' : np.NaN,
                      '0?' : np.NaN
                      }, inplace = True)

# 일 변환
crawl['일'].replace( {'0-' : np.NaN,
                      '0?' : np.NaN
                      }, inplace = True)

In [15]:
# 출시일 컬럼 생성

crawl['출시일'] = crawl['년'] + crawl['월'] + crawl['일']

In [16]:
crawl['출시일'].unique()

array(['20040601', '20040720', '20040607', '20040629', '20060420',
       '20070426', '20090416', '20160623', '20050428', '20190328',
       '20101230', '20151223', '20201112', '20070222', '20080904', nan,
       '20171221', '20041028', '20040920', '20070920', '20120329',
       '20160401', '20050414', '20071108', '20040727', '20110526',
       '20080522', '20110811', '20100916', '20110825', '20120906',
       '20121129', '20210429', '20050616', '20080117', '20100325',
       '20050504', '20051020', '20040708', '20061221', '20110210',
       '20210107', '20210401', '20040818', '20041209', '20060831',
       '20061116', '20211028', '20130822', '20050512', '20041111',
       '20110407', '20211021', '20050908', '20060914', '20070809',
       '20081113', '20041125', '20071227', '20090514', '20120202',
       '20050714', '20050825', '20061026', '20070308', '20070628',
       '20070510', '20091015', '20051017', '20071004', '20090611',
       '20060323', '20071122', '20051221', '20070208', '2

In [18]:
# 날짜형으로 변경

crawl['출시일'] = pd.to_datetime(crawl['출시일'])

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일
0,포레스트 통나무,1,3,6.0,반시계방향,2004년 6월 1일,2004,06,01,2004-06-01
1,포레스트 버섯동굴,2,2,5.2,시계방향,2004년 7월 20일,2004,07,20,2004-07-20
2,포레스트 골짜기,1,3,7.5,시계방향,2004년 6월 7일,2004,06,07,2004-06-07
3,포레스트 기암괴석,2,3,5.4,시계방향,2004년 6월 29일,2004,06,29,2004-06-29
4,포레스트 폭포속으로,2,3,4.4,시계방향,2006년 4월 20일,2006,04,20,2006-04-20
...,...,...,...,...,...,...,...,...,...,...
330,코리아 부산의 밤,3,2,8.2,반시계방향,2021년 7월 22일,2021,07,22,2021-07-22
331,코리아 제주 해오름 다운힐,2,1,5.6,단선형,2021년 8월 12일,2021,08,12,2021-08-12
332,[R] 코리아 다이나믹 서울,3,1,6.6,반시계방향,2022년 2월 17일,2022,02,17,2022-02-17
333,[R] 코리아 부산의 밤,3,2,8.2,시계방향,2022년 2월 17일,2022,02,17,2022-02-17


In [24]:
# 출시일의 nan 값 0으로 변경

crawl = crawl.fillna('0')

In [175]:
# 중복 데이터 있는지 확인
# 30개 행의 map_name이 중복되고 있음

crawl.map_name.nunique()

304

In [181]:
# 중복 내용이 모든 컬럼에 대해 중복인지 확인
# 모든 컬럼에 대한 중복이 맞음
crawl.loc[crawl.map_name.duplicated()==True].sort_values(by = 'map_name', ascending = False)

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일
58,빌리지 파괴된 시계탑,2,3,6.3,8자형,2010년 12월 30일,2010,12,30,2010-12-30 00:00:00
74,빌리지 파괴된 손가락,3,3,6.0,시계방향,2011년 8월 25일,2011,08,25,2011-08-25 00:00:00
65,빌리지 크레이지 운하,2,3,4.8,시계방향,2015년 12월 23일,2015,12,23,2015-12-23 00:00:00
75,빌리지 지우펀,4,1,5.5,단선형,2012년 9월 6일,2012,09,06,2012-09-06 00:00:00
71,빌리지 익스트림 경기장,3,2,7.4,시계방향,2008년 5월 22일,2008,05,22,2008-05-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...
63,[R] 빌리지 수로,2,3,6.0,8자형,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00
79,[R] 빌리지 손가락,3,3,6.0,반시계방향,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00
78,[R] 빌리지 붐힐터널,4,1,5.3,반시계방향,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00
61,[R] 빌리지 두개의 관문,1,3,6.9,반시계방향,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00


In [183]:
# 중복 제거
crawl = crawl.drop_duplicates()

## json 기반 trackId와 theme table merge하기

In [74]:
compare_table = pd.read_csv("C:\\Users\\wldbs\\Desktop\\kart\\00-json-metadata-trackId-with-map-name.csv", encoding = 'cp949')


# 스피드전에 대한 테마만 존재
theme_table = pd.read_csv('C:\\Users\\wldbs\\Desktop\\kart-game-project\\[data]_crawling\\track_theme.csv')

In [70]:
# compare_table.map_name.unique()

In [71]:
# print(theme_table.nunique())
# print(compare_table.nunique())

In [75]:
# 카멜롯 외곽 순찰로 이름으로 trackId가 두 개 있음 
# 확인해보니, 3일치 데이터에서는 4cdbb5 ~ trackId만 존재함
# 9f583 ~ ID는 과거의 아이디였을 것으로 추정됨
# 삭제할지, (구) 를 붙여 남겨둘지는 고민이 필요할 듯

compare_table['duplicated'] = compare_table['map_name'].duplicated()
compare_table.loc[compare_table['duplicated'] == True]

compare_table['trackId'].loc[compare_table.map_name == '카멜롯 외곽 순찰로'].to_list()

['9f5830793101bb00f5b1889745bef379020e668e8b969b53906113ec67a25a86',
 '4cdbb5c58c951f3442fe95903edba1a6b4ad8d03a1752baab3fa43a7ca78b8e8']

In [86]:
# 카멜롯 외곽 순찰로의 두 trackId 중 9f~ id는 과거 id로 판단하고, 삭제하기로 결정
# map_name 으로 join하기 때문에 (구) 를 붙여 이름을 바꾸더라도, 교집합으로 처리되지 않아 삭제될 것으로 예상되었기 때문

drop_data = compare_table[compare_table['trackId'] == '9f5830793101bb00f5b1889745bef379020e668e8b969b53906113ec67a25a86'].index

compare_table.drop(drop_data, inplace = True)

# 필요 없는 'duplicated' 컬럼 제거
compare_table = compare_table.drop(['duplicated'], axis = 1)

# 혹시를 위해 카멜롯 외곽 순찰로의 trackId 출력해봄
compare_table['trackId'].loc[compare_table.map_name == '카멜롯 외곽 순찰로'].to_list()

In [87]:
# compare_table.loc[compare_table.map_name.str.contains('카멜롯') == True]

,trackId,map_name
23,13ffa2e2685e7fd80494ab652f277ab649eefb2b2a9853...,카멜롯 불타는 마을
59,de7d51d665886f5ebe8f54095276af5b54e0c49171b6f8...,카멜롯 전차 추격전
87,2ac0e5f5bf68da7b8e2e0090d8ac7863b713448ab5994b...,카멜롯 기사단 훈련장
88,c041b6db32384707dd0ee983dc73eb90aaebf2b4271541...,카멜롯 비밀의 장소
146,458eb75fb11d035d7e2e06fa76a5dd6bb9e5639a29c3c7...,카멜롯 순찰로 역주행
158,4cdbb5c58c951f3442fe95903edba1a6b4ad8d03a1752b...,카멜롯 외곽 순찰로
291,f22f8ac7e58af894e47c2bef43860e41662216dfe8e9dc...,카멜롯 펜드래건 캐슬
306,46a9e718511d860429d6d9d984ebc0c93458c3571e7bef...,카멜롯 바다 하역장


In [158]:
# trackId_name 테이블과 theme_table merge
# 여기서 theme 나오지 않는 map_name의 경우 나중에 수기로 붙이기로 결정

new_meta_table = pd.merge(theme_table, compare_table, left_on = '트랙이름', right_on = 'map_name', how = 'right')

new_meta_table.loc[new_meta_table['트랙이름'] == new_meta_table['map_name'], 'result'] = 'equal'

In [159]:
# 저장하기 전 필요 없는 컬럼 제거

new_meta_table = new_meta_table.drop(['트랙이름', 'result'], axis = 1)

In [161]:
# 저장

new_meta_table.to_csv('C:\\Users\\wldbs\\Desktop\\kart\\1026\\map_name_with_theme.csv', index=False, encoding = 'utf-8')

## 엑셀에서 변환해온 META TABLE 재로드

In [163]:
map_name_with_theme = pd.read_csv('C:\\Users\\wldbs\\Desktop\\kart\\1026\\map_name_with_theme.csv', encoding = 'utf-8')

In [164]:
map_name_with_theme

,테마,trackId,map_name
0,빌리지,c93843288b6c038f11328c0e4a4dae0663a94154242059...,빌리지 두개의 관문
1,팩토리,c5d2203fc4979dab295e07763de827776f4e83c373cc66...,팩토리 핀저의 위험한 적재소
2,올림포스,bfd558c15104ec3ef0f182893a3c9bd77052ce7675028c...,올림포스 선택의 문
3,WKC,e86138a3a04c138f991da883d4052ff3a00624ba9eb29d...,WKC 싱가폴 마리나 서킷
4,어비스,bced4f65cf36c22df94c6e26436e37f22b1cf42b5878e2...,어비스 스카이라인
...,...,...,...
338,팩토리,ee9f7fc6ac66066247bc8de759996057ae42a2a5d2283f...,팩토리 브로디의 심술
339,문힐시티,6f39b6766c8a93b662250c86c038154139b42448d12f0c...,문힐시티 지피의 미완성 빌딩
340,월드,783f08e9fdaa169ec3138eecf72451e9439211d414e200...,월드 그리스의 휴일
341,해변,3b3d58a41efb9b00da3e88874a90b2e97ce0bc43381e7a...,비치 신나는 여름 휴가


In [188]:
merged_crwal = pd.merge(crawl, map_name_with_theme, on = 'map_name', how = 'left')

In [193]:
# 제대로 안 붙은 거 17개
# 많은 수가 아니니 엑셀로 수작업 해서 붙이기로 함

merged_crwal.loc[merged_crwal.trackId.isnull()]

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일,테마,trackId
29,빌리지 뛰라이 운하,3,3,4.8,시계방향,2016년 4월 1일,2016,04,01,2016-04-01 00:00:00,NaN,NaN
66,아이스 짜릿한 고공점프,2,2,10.1,시계방향,2021년 1월 7일,2021,01,07,2021-01-07 00:00:00,NaN,NaN
74,공동묘지 어둠의 박쥐성,2,3,6.3,워프형,2021년 10월 28일,2021,10,28,2021-10-28 00:00:00,NaN,NaN
82,유령의 축제,3,1,6.4,시계방향,2021년 10월 21일,2021,10,21,2021-10-21 00:00:00,NaN,NaN
120,팩토리 뛰라이 모험,3,3,6.3,반시계방향,2016년 4월 1일,2016,04,01,2016-04-01 00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
191,차이나 다가오는 어둠,3,1,8.7,단선형,2021년 07월 22일,2021,07,22,2021-07-22 00:00:00,NaN,NaN
199,대저택 오후의 휴일,2,2,5.6,시계방향,2020년 8월 20일,2020,08,20,2020-08-20 00:00:00,NaN,NaN
209,뱅크 트랙 (삭제),1,5,0.0,시계방향,2014년 8월 7일,2014,08,07,2014-08-07 00:00:00,NaN,NaN
244,월드 뛰라이 그리스,2,2,6.0,반시계방향,2016년 4월 1일,2016,04,01,2016-04-01 00:00:00,NaN,NaN


In [194]:
# 저장

merged_crwal.to_csv('C:\\Users\\wldbs\\Desktop\\kart\\1026\\crawl_data_with_theme.csv', index=False, encoding = 'utf-8')